ELEC-E8105 

Non-linear filtering and parameter estimation Spring 2019

Erkka Tahvanainen

K80343

Exercise round 5

# Exercise 1. (Sigma-point Methods and Linear Functions)

Show that when the function is linear, both the unscented transform and the
spherical cubature rule give the exact result.

### Unscented Transform

$w^{(i)}$ have been chosen such that original mean and variance can be recovered

$$\mu = \sum_{i} w^{(i)} \chi^{(i)}$$

$$\Sigma^ = \sum_{i} w^{(i)}(\chi^{(i)}-\mu)(\chi^{(i)}-\mu)^T$$

$$ \chi^{(0)} = m $$
$$ \chi^{(i)} = m + \sqrt{n +\lambda}[\sqrt{P}]_{i}$$

$$ \chi^{(i+n)} = m -\sqrt{n +\lambda}[\sqrt{P}]_{i}$$

Transformation $y = g(x) = Ax$

Approximations
$$\mu_y = \sum_{i} w^{(i)} g(\chi^{(i)})$$

$$\Sigma_{y} = \sum_{i} w^{(i)}(g(\chi^{(i)})-\mu_y)(g(\chi^{(i)})-\mu_y)^T$$

$$\mu_y = \sum_{i} w^{(i)} g(\chi^{(i)}) = \sum_{i} w^{(i)} A \chi^{(i)} = A \sum_{i} w^{(i)}\chi^{(i)} = A \mu$$



$$\Sigma_{y} = \sum_{i} w^{(i)}(g(\chi^{(i)})-\mu_y)(g(\chi^{(i)})-\mu_y)^T = \sum_{i} w^{(i)}(A \chi^{(i)}-A \mu)(A \chi^{(i)}-A\mu)^T$$

$$  = \sum_{i} w^{(i)}A(\chi^{(i)}-\mu)(\chi^{(i)}-\mu)^TA^T = A\sum_{i} w^{(i)}(\chi^{(i)}-\mu)(\chi^{(i)}-\mu)^TA^T = A\Sigma A^T$$

### Spherical cubature

$$ \xi^{(i)}  =  \begin{cases} 
      \sqrt{n}e_i & i = 1,...,n \\
      -\sqrt{n}e_{i-n} & i=n+1,....,2n 
   \end{cases}$$ where $e_i$ is unit vector pointing direction of axis i.

$$ \int g(x) N(x|m,P)dx = \int g(m + \sqrt{P} \xi) N(\xi|0,I)d\xi \approx \frac{1}{2n} \sum_{i=1}^{2n} g(m+\sqrt{P}\xi^{(i)})$$

$$\frac{1}{2n} \sum_{i=1}^{2n} g(m+\sqrt{P}\xi^{(i)}) = \frac{1}{2n} \sum_{i=1}^{2n} (Am+A\sqrt{P}\xi^{(i)}) = \frac{1}{2n} (Am*2n + \sum_{i=1}^{2n} A\sqrt{P}\xi^{(i)}) = Am + \frac{1}{2n}\sum_{i=1}^{2n} A\sqrt{P}\xi^{(i)} = Am$$

$$ \int g(x) N(x|m,P)dx = \int g(m + \sqrt{P} \xi) N(\xi|0,I)d\xi  = \int (Am + A\sqrt{P} \xi) N(\xi|0,I)d\xi$$

$$ = Am \int N(\xi|0,I)d\xi + A\sqrt{P}\int \xi N(\xi|0,I)d\xi  = Am + A\sqrt{P}\int \xi N(\xi|0,I)d\xi = Am + A\sqrt{P} E[\xi]   = Am$$

# Exercise 2. (Unscented Kalman filter)

Consider the following non-linear state space model

$$ x_k = x_{k-1} - 0.01 sin (x_{k-1}) + q_{k_1}$$

$$y_k = 0.5 sin (2x_k) + r_k$$

where $q_{k-1}$ has variance $0.01^2$ and $r_k$ has variance $0.02$.


Derive and implement UKF to the model in Exercise 1 of Round 4, that
is, to the same problem where you implemented EKF. Implement the UKF
equations yourself, i.e., do not use the EKF/UKF toolbox or similar.

In [ ]:
import numpy as np
import math
from sklearn.metrics import mean_squared_error

In [ ]:
def f(x):
    return x -0.01*math.sin(x)
def h(x):
    return 0.5*math.sin(2*x)
    

In [ ]:
def df(x):
    return 1-0.01*math.cos(x)
def dh(x):
    return math.cos(2*x)

In [ ]:
import scipy.io as sio
from filterpy.kalman import UnscentedKalmanFilter,ExtendedKalmanFilter,MerweScaledSigmaPoints
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from numpy.random import normal
import seaborn as sns
from scipy.linalg import cholesky
%matplotlib notebook

In [ ]:
np.random.seed(123)
n = 100
q_sd = 0.01
#I assume here that in exercise there was mistake as sqrt(0.02)=0.14 which is very high...
r_sd = 0.02
x_meas = np.zeros(n)
y_meas = np.zeros(n)
x_true = np.zeros(n)
y_true = np.zeros(n)
x_true[0] = 0
y_true[0] = h(x_true[0])
for j in range(1,n):
    x_meas[j] = f(x_true[j-1]) + normal(0,q_sd)
    y_true[j] = h(x_meas[j])
    #I assume here that 
    y_meas[j] = y_true[j] + normal(0,r_sd)

# Unscented Kalman filter

In [ ]:
#EKF for reference

EKF_EST = np.zeros(len(y_meas))
mk = np.array([f(0)])
Pk = np.array([0.01**2])
for k in range(n):
    #Prediction
    mk_ =  np.array([[f(mk)]])
    F = np.array([[df(mk)]])
    Pk_ = F @ np.array([Pk])@ F.T + q_sd**2
    #Update
    v_k = np.array([[y_meas[k] - h(mk_)]])
    H = np.array([[dh(mk_)]])
    S_k = H@Pk_@H.T + r_sd**2
    K_k = Pk_@H.T @ np.linalg.inv(S_k)
    mk = (mk_ + K_k @ v_k).flatten()
    Pk = (Pk_ - K_k @S_k@K_k).flatten()
    EKF_EST[k] = mk

In [ ]:
def fx(x, dt):
    return np.array([f(x)])
def hx(x):
# measurement function - convert state into a measurement
# where measurements are [x_pos, y_pos]
    return np.array([h(x)])

dt = 0.1
# create sigma points to use in the filter. This is standard for Gaussian processes
points = MerweScaledSigmaPoints(1, alpha=1, beta=0, kappa=0)
kf = UnscentedKalmanFilter(dim_x=1, dim_z=1, dt=dt, fx=fx, hx=hx, points=points)
kf.x = np.array([0])# initial state
kf.P *= q_sd**2 # initial uncertainty
kf.R = np.diag([r_sd**2]) # 1 standard
kf.Q = np.array([q_sd**2])
UKF_EST = np.zeros(len(y_meas))

for idx, z in enumerate(y_meas):
    kf.predict()
    kf.update(z)
    UKF_EST[idx] = kf.x
#    print(kf.x, 'log-likelihood', kf.log_likelihood)

In [ ]:
t = np.linspace(1,len(y_meas),len(y_meas))
plt.figure(figsize=(20,10))
plt.suptitle("Trajectories and EKF estimates")
plt.plot(t,y_true,label = "True signal",linestyle='--',color='b',alpha=0.5)
plt.scatter(t,y_meas, 
                 label = "Baseline RMSE ={:0.5f}".format(np.sqrt(mean_squared_error(y_true,y_meas))),
                   s=80, facecolors='none', edgecolors='r',alpha=0.5)
plt.plot(t,EKF_EST, 
                 label = "EKF estimate RMSE ={:0.5f}".format(np.sqrt(mean_squared_error(y_true,EKF_EST)))
                ,linestyle='-',color='r',alpha=0.5)
plt.plot(t,UKF_EST, 
                 label = "UKF estimate RMSE ={:0.5f}".format(np.sqrt(mean_squared_error(y_true,UKF_EST)))
                ,linestyle='-',color='g',alpha=0.5)
plt.legend()
plt.show()

In [ ]:
# Precompute the UT weights
n = 1;
alpha = 1;
beta = 0;
#   kappa = 3-n;
kappa = 0;

lambda_ = alpha**2 * (n + kappa) - n;        
WM = np.zeros(2*n+1);
WC = np.zeros(2*n+1);
for j in range(0,2*n+1):
    if j==0:
        wm = lambda_ / (n + lambda_);
        wc = lambda_ / (n + lambda_) + (1 - alpha**2 + beta);
    else:
        wm = 1 / (2 * (n + lambda_));
        wc = wm;
    
    WM[j] = wm;
    WC[j] = wc;


In [ ]:
m = f(0)
P = 0.01**2

In [ ]:
Ms = []
Ps = []
multipliers = [np.sqrt(n+lambda_) for i in range(0,n)]+[-np.sqrt(n+lambda_) for i in range(0,n)]
for inx,val in enumerate(y_meas):
    A = np.sqrt(P)
    sigma_points = np.zeros(len(WM))
    sigma_y = np.zeros(len(WM))
    sigma_f = np.zeros(len(WM))
    #calculate sigma points around m with f
    sigma_points[0] = m
    sigma_f[0] = h(sigma_points[0])
    for i in range(1,len(sigma_points)):
        sigma_points[i] = m + multipliers[i-1]*A #A is scalar
        sigma_f[i] = f(sigma_points[i])
    mk_ = 0
    Pk_ = 0
    
    #prediction step
    for i in range(len(sigma_points)):
        mk_ = mk_ + WM[i]*sigma_f[i]
    for i in range(len(sigma_points)):
        Pk_ = Pk_ + WC[i] * (sigma_f[i]  - mk_) * (sigma_f[i] - mk_) + q_sd**2
        
    #update step
    A = np.sqrt(Pk_)
    
    #calculate sigma points around mk_ with h
    sigma_points[0] = mk_
    sigma_y[0] = h(sigma_points[0])
    for i in range(1,len(sigma_points)):
            sigma_points[i] = mk_ + multipliers[i-1]*A #A is scalar
            sigma_y[i] = h(sigma_points[i])
    
    mu_k = 0
    S_k = 0
    C_k  = 0
    for i in range(len(sigma_points)):
        mu_k = mu_k + WM[i]*sigma_y[i]
    for i in range(len(sigma_points)):
        S_k = S_k + WC[i] * (sigma_y[i]  - mu_k) * (sigma_y[i] - mu_k) + r_sd**2
        C_k = C_k + WC[i]* (sigma_points[i]  - mk_) * (sigma_y[i] - mu_k) 
    K_k = C_k/S_k
    m = mk_ + K_k*(val - mu_k)
    P = Pk_ - K_k*S_k*K_k
    Ms.append(m)
    Ps.append(P)

In [ ]:
plt.figure(figsize=(20,10))
plt.suptitle("Trajectories and EKF estimates")
plt.plot(t,y_true,label = "True signal",linestyle='--',color='b',alpha=0.5)
plt.scatter(t,y_meas, 
                 label = "Baseline RMSE ={:0.5f}".format(np.sqrt(mean_squared_error(y_true,y_meas))),
                   s=80, facecolors='none', edgecolors='r',alpha=0.5)
plt.plot(t,Ms, 
                 label = "UKF estimate RMSE ={:0.5f}".format(np.sqrt(mean_squared_error(y_true,Ms)))
                ,linestyle='-',color='r',alpha=0.5)
plt.legend()
plt.show()

# Exercise 3. (Bearings Only Target Tracking with CKF and UKF)

The state of the target at time step $k$ consist of the position $(x_k, y_k)$ and the
velocity $(\dot x_k,\dot y_k)$. The dynamics of the state vector $x_k = (x_k, y_k, \dot x_k, \dot y_k)^T$ are
modeled with the discretized Wiener velocity model:

$$  \begin{pmatrix}
  x_k  \\
  y_k \\
 \dot x_k\\
 \dot y_k \\
  \end{pmatrix} = \begin{pmatrix} 
 1 & 0 & \Delta t & 0 \\
 0 & 1 & 0 & \Delta t \\
 0 & 0 & 1 & 0 \\
 0 & 0 & 0 & 1 
  \end{pmatrix}\begin{pmatrix}
  x_{k-1}  \\
  y_{k-1} \\
 \dot x_{k-1}\\
 \dot y_{k-1}
  \end{pmatrix} + q_{k-1}$$

where $q_k$ is a zero mean Gaussian process noise with covariance

$$Q = \begin{pmatrix} 
 q_1^c \Delta t^3/3 & 0 & q_1^c\Delta t^2/2 & 0 \\
 0 & q_2^c \Delta t^3/3 & 0 & q_2^c \Delta t^2 /2\\
 q_1^c \Delta t^2/2 & 0 & q_1^c \Delta t & 0 \\
 0 & q_2^c \Delta t^2/2 & 0 & q_2^c \Delta t 
  \end{pmatrix}$$
  
  
In this scenario the diffusion coefficients are $q_c^1 = q_c^2 = 0.1$ and the sampling period is $\Delta t = 0.1$. The measurement model for sensor $i \in \{1, 2\}$ is the following:

$$\theta_k^i = tan^{-1} \big(\frac{y_k - s_y^i}{x_k - s_x^i}\big) +r_k$$

where $(s_x^i,s_y^i)$ is the position of the sensor $i$ and $r_k \sim N(0, \sigma^2)$ is a Gaussian
measurement noise with standard deviation of $\sigma = 0.05$ radians. At each sampling time, which occurs 10 times per second (i.e., $\Delta t = 0.1$), both of the two sensors produce a measurement.

In the file angle_ex.m (in MyCourses) there is a baseline solution, which
computes estimates of the position from the crossing of the measurements
and estimates the velocity to be always zero. Your task is to implement an
EKF for the problem and compare the results graphically and in RMSE
sense.

In [ ]:
#load simulated data.
Theta = sio.loadmat('Theta.mat')['Theta']
Theta = Theta.swapaxes(0,1)
X = sio.loadmat('X.mat')['X']
X = X.swapaxes(0,1)

In [ ]:
steps = Theta.shape[0]
x0 = np.array([[0],[0],[1],[0]])
S1 = np.array([[-1.5],[0.5]])# Position of sensor 1
S2 = np.array([[1],[1]])      # Position of sensor 2


In [ ]:
qc = 0.1
dt = 0.01
sd = 0.05;
#This is the transition matrix
A = np.array([[1,0,dt,0],
             [0,1,0,dt],
             [0,0,1,0],
             [0,0,0,1]])
#This is the process noise covariance
Q = np.array([[qc*dt**3/3,0, qc*dt**2/2, 0],
       [0, qc*dt**3/3, 0, qc*dt**2/2],
       [qc*dt**2/2, 0, qc*dt, 0],
       [0, qc*dt**2/2, 0, qc*dt]])

In [ ]:
# Initialize to true value
m1 = x0
EST1 = np.zeros((steps,4))
  

# Loop through steps
for k in range(steps):
    # Compute crossing of the measurements
    dx1 = np.cos(Theta[k,0])
    dy1 = np.sin(Theta[k,0])
    dx2 = np.cos(Theta[k,1])
    dy2 = np.sin(Theta[k,1])
    d = np.linalg.solve(np.array([[dx1, dx2],[dy1, dy2]]),np.array([S2[0]-S1[0],S2[1]-S1[1]]))
    # Crossing
    cross_xy = S1 + np.array([[dx1],[dy1]])*d[0]
    # compute estimate
    m1 = np.concatenate([cross_xy.flatten(),[0],[0]])
    EST1[k,:] = m1  

In [ ]:
def rmse(X1,X2):
    return np.sqrt(np.mean(np.sum((X1-X2)**2,axis=1)))

# Baseline solution

Animation

In [ ]:
from matplotlib import animation, rc

In [ ]:
def animate_estimate(EST,title_sol):
    rc('animation', embed_limit=41030228 )
    fig1,ax = plt.subplots(figsize=(20,10))
    ax.set_title('{} solution'.format(title_sol))
    ax.set_xlim([-2,2])
    ax.set_ylim([-2.5,1.5])
    ax.plot(X[:,0],X[:,1],linestyle = '--',color='b',alpha=0.5,label='True trajectory')
    ax.scatter(S1[0],S1[1],marker = 'X',s=80, facecolors='none', edgecolors='k',label='Sensor 1')
    ax.scatter(S2[0],S2[1],s=80, facecolors='none', edgecolors='k',label='Sensor 2')
    l, = ax.plot([],[],linestyle = '-',color='r',alpha=0.5,label='{} estimate RMSE = {:0.3f}'.format(title_sol,rmse(X,EST)))
    o, = ax.plot([],[],marker = 'o', color='k')
    s_1, = ax.plot([],[],linestyle = '--',color='k')
    s_2, = ax.plot([],[],linestyle = '--',color='k')
    ax.legend()

    def animate(i):
        l.set_data(EST[:i+1,0],EST[:i+1,1])
        o.set_data(EST[i:i+1,0],EST[i:i+1,1])
        len = 4 
        dx1 = len*np.cos(Theta[i,0]);
        dy1 = len*np.sin(Theta[i,0]);
        dx2 = len*np.cos(Theta[i,1]);
        dy2 = len*np.sin(Theta[i,1]);
        s_1.set_data([S1[0],S1[0]+dx1],[S1[1],S1[1]+dy1])
        s_2.set_data([S2[0],S2[0]+dx2],[S2[1],S2[1]+dy2])
        return (l,o,s_1,s_2)
    def init():
        l.set_data([],[])
        o.set_data([],[])
        s_1.set_data([],[])
        s_2.set_data([],[])
        return (l,o,s_1,s_2)

    return animation.FuncAnimation(fig1, animate, init_func=init,frames=len(EST))

In [ ]:
%%capture
ani = animate_estimate(EST1,"Baseline")

In [ ]:
#from IPython.display import HTML
#HTML(ani.to_jshtml())

In [ ]:
def plot_estimate(EST,title_sol):
    plt.figure(figsize=(20,10))
    plt.title('{} solution'.format(title_sol))
    plt.xlim([-2,2])
    plt.ylim([-2.5,1.5])
    plt.plot(X[:,0],X[:,1],linestyle = '--',color='b',alpha=0.5,label='True trajectory')
    plt.plot(EST[:,0],EST[:,1],linestyle = '-',color='r',alpha=0.5,label='{} estimate RMSE = {:0.3f}'.format(title_sol,rmse(X,EST)))
    plt.scatter(S1[0],S1[1],marker = 'X',s=80, facecolors='none', edgecolors='k',label='Sensor 1')
    plt.scatter(S2[0],S2[1],s=80, facecolors='none', edgecolors='k',label='Sensor 2')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.show()

In [ ]:
plot_estimate(EST1,"Baseline")

# EKF



 $$\frac{d arctan(x)}{dx} = \frac{1}{1+x^2}$$
 
 $$ \frac{f(g(x))}{dx} = \frac{df(g(x))}{dx}\frac{g(x)}{dx}$$

In [ ]:
def darctan(x):
    return 1.0/(1+x**2)

def g(mk,s):
    return (mk[1,0]-s[1,0])/(mk[0,0]-s[0,0])

def derivative_x(mk,s):
    return (s[1,0]-mk[1,0])/(s[0,0]**2 - 2*s[0,0]*mk[0,0]+s[1,0]**2 - 2 *s[1,0]*mk[1,0] + mk[0,0]**2 + mk[1,0]**2)

def derivative_y(mk,s):
    return (mk[0,0]-s[0,0])/(s[0,0]**2 - 2.0*s[0,0]*mk[0,0]+s[1,0]**2 - 2 *s[1,0]*mk[1,0] + mk[0,0]**2 + mk[1,0]**2)
    
def dgy(mk,s):
    return 1.0/(mk[0,0]-s[0,0])
def dgx(mk,s):
    return -(mk[1,0]-s[1,0])/(mk[0,0]-s[0,0])**2
def h(mk,s):
    return np.arctan2(mk[1,0]-s[1,0],mk[0,0]-s[0,0])

In [ ]:
m2 = x0           # Initialize to true value
P2 = np.eye(4)        # Some uncertainty
R  = sd**2*np.eye(2)   # The joint covariance
EST2 = np.zeros((steps,4))

for k in range(steps):
    #predict
    mk_ = A @ m2
    Pk_ = A @ P2 @A.T + Q
    #update
    v_k = (Theta[k] - np.array([[h(mk_,S1),h(mk_,S2)]])).T
    #H  = np.array([[darctan(g(mk_,S1))*dgx(mk_,S1),darctan(g(mk_,S1))*dgy(mk_,S1),0,0],
    #               [darctan(g(mk_,S2))*dgx(mk_,S2),darctan(g(mk_,S2))*dgy(mk_,S2),0,0]])
    H = np.array([[derivative_x(mk_,S1), derivative_y(mk_,S1),0,0],
                 [derivative_x(mk_,S2),derivative_y(mk_,S2),0,0]])
    S_k = H@Pk_@H.T + R
    K_k = Pk_@H.T @ np.linalg.inv(S_k)
    m2 = mk_ + K_k @ v_k
    P2 = Pk_ - K_k @ S_k @ K_k.T
    EST2[k,:] = m2.flatten()

In [ ]:
%%capture
ani = animate_estimate(EST2,"EKF")

In [ ]:
#from IPython.display import HTML
#HTML(ani.to_jshtml())

In [ ]:
plot_estimate(EST2,"EKF")

## Unscented Kalman filter using filterpy

In [ ]:
x0

In [ ]:
def fx(x, dt):
    return np.dot(A,x)
def hx(x):
    return  np.array([h(x.reshape(4,1),S1),h(x.reshape(4,1),S2)])
    
dt = 0.1

# create sigma points to use in the filter. This is standard for Gaussian processes
points = MerweScaledSigmaPoints(4, alpha=1, beta=0, kappa=0)
kf = UnscentedKalmanFilter(dim_x=4, dim_z=2, dt=dt, fx=fx, hx=hx, points=points)
kf.x = x0.flatten()# initial state
kf.P = np.eye(4) # initial uncertainty
kf.R = R # 1 standard
kf.Q = Q
UKF_EST = np.zeros((len(Theta),4))

for idx, z in enumerate(Theta):
    kf.predict()
    kf.update(z.flatten())
    UKF_EST[idx] = kf.x

In [ ]:
plot_estimate(UKF_EST,"UKF filterpy")

In [ ]:
# Precompute the UT weights
n = 4;
alpha = 1;
beta = 0;
#   kappa = 3-n;
kappa = 0;

lambda_ = alpha**2 * (n + kappa) - n;        
WM = np.zeros(2*n+1);
WC = np.zeros(2*n+1);
for j in range(0,2*n+1):
    if j==0:
        wm = lambda_ / (n + lambda_);
        wc = lambda_ / (n + lambda_) + (1 - alpha**2 + beta);
    else:
        wm = 1 / (2 * (n + lambda_));
        wc = wm;
    
    WM[j] = wm;
    WC[j] = wc;

In [ ]:
def hx(x):
# measurement function - convert state into a measurement
# where measurements are [x_pos, y_pos]
    return np.array([[h(x,S1)],[h(x,S2)]])

In [ ]:
m = x0          # Initialize to true value
dim_x = 4
dim_y = 2
P = np.eye(dim_x)        # Some uncertainty
Ms = []
Ps = []
multipliers = [np.sqrt(n+lambda_) for i in range(0,n)]+[-np.sqrt(n+lambda_) for i in range(0,n)]
for inx,val in enumerate(Theta):
    #print("loop")
    P_SQRT = cholesky(P,lower=True)
    sigma_points = np.zeros((len(WM),dim_x,1))
    sigma_y = np.zeros((len(WM),dim_y,1))
    sigma_f = np.zeros((len(WM),dim_x,1))
    #calculate sigma points around m with f
    sigma_points[0] = m
    sigma_f[0] = np.dot(A,sigma_points[0])
    for i in range(1,len(sigma_points)):
        sigma_points[i] = m + multipliers[i-1]*P_SQRT[:,(i-1)%dim_x].reshape(dim_x,1)
        sigma_f[i] = np.dot(A,sigma_points[i])
   
    mk_ = np.zeros((dim_x,1))
    Pk_ = np.zeros((dim_x,dim_x)) 

    #prediction step
    mk_ = sigma_f.squeeze().T.dot(WM).reshape((dim_x,1))
    points_diff_f = sigma_f.squeeze().T - mk_
    Pk_ = points_diff_f.dot(np.diag(WC)).dot(points_diff_f.T) + Q
   
    #print(sigma_points.reshape(4,9)@WM)
   
    #update step

    P_SQRT = cholesky(Pk_,lower=True)
   
    #calculate sigma points around mk_ with h
    sigma_points[0] = mk_
    sigma_y[0] = hx(sigma_points[0])
    
    for i in range(1,len(sigma_points)):
            sigma_points[i] = mk_ + multipliers[i-1]*P_SQRT[:,(i-1)%dim_x].reshape(dim_x,1) 
            sigma_y[i] = hx(sigma_points[i])
            
    mu_k = np.zeros((dim_y,1))
    S_k = np.zeros((dim_y,dim_y))
    C_k  = np.zeros((dim_x,dim_y))
    
    mu_k = sigma_y.squeeze().T.dot(WM).reshape((dim_y,1))
    points_diff_y = sigma_y.squeeze().T - mu_k
    S_k = points_diff_y.dot(np.diag(WC)).dot(points_diff_y.T) + R
    C_k = points_diff_f.dot(np.diag(WC)).dot(points_diff_y.T)
    
    K_k = C_k@np.linalg.inv(S_k)
    m = mk_ + K_k@(val.reshape(dim_y,1) - mu_k)
    P = Pk_ - K_k@S_k@K_k.T
    
    Ms.append(m)
    Ps.append(P)
EST_UKF = np.array(Ms).squeeze()

In [ ]:
%%capture
ani = animate_estimate(EST_UKF,"UKF")

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
plot_estimate(EST_UKF,"UKF")

# Cubature Kalman Filter

In [ ]:
n = 4
multipliers = [np.sqrt(n) for i in range(0,n)]+[-np.sqrt(n) for i in range(0,n)]
unit_vectors = np.array([[[1],[0],[0],[0]],[[0],[1],[0],[0]],[[0],[0],[1],[0]],[[0],[0],[0],[1]]])


In [ ]:
m = x0          # Initialize to true value
dim_x = 4
dim_y = 2
P = np.eye(dim_x)        # Some uncertainty
Ms = []
Ps = []
n = 4
multipliers = [np.sqrt(n) for i in range(0,n)]+[-np.sqrt(n) for i in range(0,n)]
unit_vectors = np.array([[[1],[0],[0],[0]],[[0],[1],[0],[0]],[[0],[0],[1],[0]],[[0],[0],[0],[1]]])
for inx,val in enumerate(Theta):
    P_SQRT = cholesky(P,lower=True)
    sigma_points = np.zeros((len(multipliers),dim_x,1))
    sigma_y = np.zeros((len(multipliers),dim_y,1))
    sigma_f = np.zeros((len(multipliers),dim_x,1))
    #calculate sigma points around m with f
    for i in range(0,len(sigma_points)):
        sigma_points[i] = m + multipliers[i]*P_SQRT@unit_vectors[i%n]
        sigma_f[i] = np.dot(A,sigma_points[i])
      #prediction step
    mk_ = np.sum(sigma_f,axis=0)/(2*n)

    
    points_diff_f = (sigma_f-mk_).squeeze().T
    Pk_ = points_diff_f.dot(points_diff_f.T)/(2*n) + Q
    #update step
    
    P_SQRT = cholesky(Pk_,lower=True)
   
    #calculate sigma points around mk_ with h
    
    for i in range(0,len(sigma_points)):
        sigma_points[i] = mk_ + multipliers[i]*P_SQRT@unit_vectors[i%n]
        sigma_y[i] =  hx(sigma_points[i])
        
            
    mu_k =  np.sum(sigma_y,axis=0)/(2*n)

    points_diff_y = (sigma_y-mu_k).squeeze().T
    S_k = points_diff_y.dot(points_diff_y.T)/(2*n) + R
    C_k = points_diff_f.dot(np.diag(np.repeat(1/(2.0*n),len(sigma_points)))).dot(points_diff_y.T)
  
    
    K_k = C_k@np.linalg.inv(S_k)
    m = mk_ + K_k@(val.reshape(dim_y,1) - mu_k)
    P = Pk_ - K_k@S_k@K_k.T

    Ms.append(m)
    Ps.append(P)
EST_CKF = np.array(Ms).squeeze()

In [ ]:
%%capture
ani = animate_estimate(EST_CKF,"CKF")

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
plot_estimate(EST_CKF,"CKF")